In [23]:
## Download the human genome in compressed FASTA format (.fa.gz.)
!wget ftp://ftp.ensembl.org/pub/release-110/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

--2025-10-21 03:24:46--  ftp://ftp.ensembl.org/pub/release-110/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
           => ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-110/fasta/homo_sapiens/dna ... done.
==> SIZE Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz ... 881964081
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz ... done.
Length: 881964081 (841M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>] 841.11M   193MB/s    in 8.2s    

2025-10-21 03:24:55 (103 MB/s) - ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’ saved [881964081]



In [24]:
# unzip the .gz file to get a readable .fa file
!gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

gzip: Homo_sapiens.GRCh38.dna.primary_assembly.fa already exists; do you wish to overwrite (y or n)? y


In [11]:
## Install necessary libraries to read and parse the FASTA files
%%capture
! pip install Biopython

In [25]:
from Bio import SeqIO # SeqIO module is use to read and write the biological sequence files ( FASTA,GenBank)
# Fetch the records in the FASTA
for record in SeqIO.parse("Homo_sapiens.GRCh38.dna.primary_assembly.fa", "fasta"):
    print(f'chromosome ID: {record.id}')     # each record corresponds to an identifier chromosome or scaffold in human genome ( "1","2","X","Y","MT" etc)
    print(f'full ID description :{record.description}') #full ensemble style header
    print(record.seq[:100])
    print(f'sequence length: {len(record.seq)}')
    print('\n')


chromosome ID: 1
full ID description :1 dna:chromosome chromosome:GRCh38:1:1:248956422:1 REF
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
sequence length: 248956422


chromosome ID: 10
full ID description :10 dna:chromosome chromosome:GRCh38:10:1:133797422:1 REF
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
sequence length: 133797422


chromosome ID: 11
full ID description :11 dna:chromosome chromosome:GRCh38:11:1:135086622:1 REF
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
sequence length: 135086622


chromosome ID: 12
full ID description :12 dna:chromosome chromosome:GRCh38:12:1:133275309:1 REF
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
sequence length: 133275309


chromosome ID: 13
full ID description :13 dna:chromosome chromosome:GRCh38:13:1:114364328:1 REF
NNN

This shows GRCh38, human genome contains many unplaced genomic scaffolds. They are a DNA sequence which is part of the genome, but has not been confidently assigned to a specific location on any chromosome. These exist because some regions in the genome are hard to assemble due to repetitive sequences, structural complpexity and low sequence coverage. If analyzing novel gene discovery or strucural variations, these scaffolds are worth exploring. But since the project goal is to analyze the DNA sequence,canonical chromosomes  will be filtered.


In [26]:
#Filter canonical chromosomes
## Generatre a clean FASTA file with 24 main chromosomes



canonical_ids = [str(i) for i in range(1, 23)] + ["X", "Y", "MT"]


# Open the full genome FASTA file
input_file = "Homo_sapiens.GRCh38.dna.primary_assembly.fa"
output_file = "canonical_chromosomes.fa"


with open(output_file,"w") as output_handle:
    for record in SeqIO.parse(input_file, "fasta"):
        if record.id in canonical_ids:
            SeqIO.write(record, output_handle, "fasta")


# Create a dictionary to store filtered sequences
filtered_sequences = {}

# Iterate and filter
for record in SeqIO.parse(input_file, "fasta"):
    chrom_id = record.id.split()[0]  # Clean ID
    if chrom_id in canonical_ids:
        filtered_sequences[chrom_id] = record.seq
        print(f"Loaded chromosome {chrom_id} with {len(record.seq)} bases.")


Loaded chromosome 1 with 248956422 bases.
Loaded chromosome 10 with 133797422 bases.
Loaded chromosome 11 with 135086622 bases.
Loaded chromosome 12 with 133275309 bases.
Loaded chromosome 13 with 114364328 bases.
Loaded chromosome 14 with 107043718 bases.
Loaded chromosome 15 with 101991189 bases.
Loaded chromosome 16 with 90338345 bases.
Loaded chromosome 17 with 83257441 bases.
Loaded chromosome 18 with 80373285 bases.
Loaded chromosome 19 with 58617616 bases.
Loaded chromosome 2 with 242193529 bases.
Loaded chromosome 20 with 64444167 bases.
Loaded chromosome 21 with 46709983 bases.
Loaded chromosome 22 with 50818468 bases.
Loaded chromosome 3 with 198295559 bases.
Loaded chromosome 4 with 190214555 bases.
Loaded chromosome 5 with 181538259 bases.
Loaded chromosome 6 with 170805979 bases.
Loaded chromosome 7 with 159345973 bases.
Loaded chromosome 8 with 145138636 bases.
Loaded chromosome 9 with 138394717 bases.
Loaded chromosome MT with 16569 bases.
Loaded chromosome X with 156040

## Extract Tumor supressor genes ((e.g., TP53, BRCA1, BRCA2, RB1, PTEN, CDKN2A) from canonical chromosomes using reference FASTA and GTF files.


In [32]:
## Parse GTF and filter for the target genes

canonical_chroms = [str(i) for i in range(1, 23)] + ["X", "Y", "MT"]

import pandas as pd

def load_gtf(gtf_path,target_genes):
    # Use the 'comment' parameter to skip lines starting with '#'
    gtf= pd.read_csv(gtf_path, sep='\t', header=None, comment='#',dtype=str)
    gtf.columns = ['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

    # Filter for canonical chromosomes
    gtf=gtf[gtf['chrom'].isin(canonical_chroms)]

    # extract the gene name
    gtf['gene_name']=gtf['attribute'].str.extract('gene_name "([^"]+)"')

    # return target genes
    return gtf[(gtf["feature"] == "gene") & (gtf["gene_name"].isin(target_genes))]

In [30]:
## Load FASTA and extract the sequences

from Bio import SeqIO # SeqIO module is use to read and write the biological sequence files ( FASTA,GenBank)

def extract_sequences(fasta_path,gtf_filtered):
  fasta_dict=SeqIO.to_dict(SeqIO.parse(fasta_path,"fasta"))
  gene_seqs={}

  for _,row in gtf_filtered.iterrows():
    chrom=row['chrom'] #sequence
    start=int(row['start'])-1  #0 based inidexing
    end=int(row['end'])
    strand=row['strand']
    gene=row['gene_name']

    seq=fasta_dict[chrom].seq[start:end] # extract the DNA sequence at the specific location
    if strand=='-':                     # if the gene is in negatuve strand, it reverses and complements the sequece
      seq=seq.reverse_complement()       ### Biologicaly necessary( Transcription reads templpate strand reverse)
    gene_seqs[gene]=str(seq)
  return gene_seqs

In [33]:
## extracting tumor supressor genes

target_genes = ["TP53", "BRCA1", "BRCA2", "RB1", "PTEN", "CDKN2A"]

gtf_filtered = load_gtf("Homo_sapiens.GRCh38.110.gtf", target_genes)
gene_seqs = extract_sequences("Homo_sapiens.GRCh38.dna.primary_assembly.fa", gtf_filtered)

# Save sequences
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO import write

records = [SeqRecord(seq, id=gene, description="") for gene, seq in gene_seqs.items()]
write(records, "tumor_suppressors.fa", "fasta")


/usr/local/lib/python3.12/dist-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


6

In [34]:
with open("tumor_suppressors.fa") as f:
    print(f.read(500))

>CDKN2A
GGTTAAAACCGAAAATAAAAATGGGCTAGACACAAAGGACTCGGTGCTTGTCCCAGCCAG
GCGCCCTCGGCGACGCGGGCAGCTGGGAGGGGAATGGGCGCCCGGACCCAGCTGGGACCC
CCGGGTGCGACTCCACCTACCTAGTCCGGCGCCAGGCCGGGTCGACAGCTCCGGCAGCGC
CAGCGCCGCGCCGTGTCCAGATGTCGCGTCAGAGGCGTGCAGCGGTTTAGTTTAATTTCG
CTTGTTTTCCAAATCTAGAAGAGGAGCGGAGCGGCTTTTAGTTCAAAACTGACATTCAGC
CTCCTGATTGGCGGATAGAGCAATGAGATGACCTCGCTTTCCTTTCTTCCTTTTTCATTT
TTAAATAATCTAGTTTGAAGAATGGAAGACTTTCGACGAGGGGAGCCAGGAATAAAATAA
GGGGAATAGGGGAGCGGGGACGCGAGCAGCACCAGAATCCGCGGGAGCGCGGCTGTTCCT
GGTA


In [36]:
## Display the tumor supressor genes

from Bio import SeqIO

# Load and display each sequence record
for record in SeqIO.parse("tumor_suppressors.fa", "fasta"):

    print(f"ID: {record.id}")
    print(f"Sequence: {record.seq[:100]}...")  # Print first 100 bases
    print(f"Length: {len(record.seq)}\n")


ID: CDKN2A
Sequence: GGTTAAAACCGAAAATAAAAATGGGCTAGACACAAAGGACTCGGTGCTTGTCCCAGCCAGGCGCCCTCGGCGACGCGGGCAGCTGGGAGGGGAATGGGCG...
Length: 27550

ID: PTEN
Sequence: GCGGGGTGCGTCCCACTCACAGGGATCCTCTTTCAGTTCATTTAGATAGGTGCCCTTTGGGCCCTTGAAATTCAACGGCTATGTGTTCACGTTCAGCACG...
Length: 109293

ID: RB1
Sequence: GGCGCTCAGTTGCCGGGCGGGGGAGGGCGCGTCCGGTTTTTCTCAGGGGACGTTGAAATTATTTTTGTAACGGGAGTCGGGAGAGGACGGGGCGTGCCCC...
Length: 295693

ID: BRCA2
Sequence: AAGCTTTTGTAAGATCGGCTCGCTTTGGGGAACAGGTCTTGAGAGAACATCCCTTTTAAGGTCAGAACAAAGGTATTTCATAGGTCCCAGGTCGTGTCCC...
Length: 85183

ID: BRCA1
Sequence: AAAGCGTGGGAATTACAGATAAATTAAAACTGTGGAACCCCTTTCCTCGGCTGCCGCCAAGGTGTTCGGTCCTTCCGAGGAAGCTAAGGCCGCGTTGGGG...
Length: 125951

ID: TP53
Sequence: GTTTTCCCCTCCCATGTGCTCAAGACTGGCGCTAAAAGTTTTGAGCTTCTCAAAAGTCTAGAGCCACCGTCCAGGGAGCAGGTAGCTGCTGGGCTCCGGG...
Length: 25760

